In [1]:
%load_ext autoreload
%autoreload 2

# Testing _crossfit Function

In [2]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from econml._ortho_learner import _crossfit

class Wrapper:
    def __init__(self, model):
        self._model = model
    def fit(self, X, y, W=None):
        self._model.fit(X, y)
        return self
    def predict(self, X, y, W=None):
        return self._model.predict(X)
np.random.seed(123)
X = np.random.normal(size=(5000, 3))
y = X[:, 0] + np.random.normal(size=(5000,))
folds = list(KFold(2).split(X, y))
model = Lasso(alpha=0.01)
nuisance, model_list, fitted_inds = _crossfit(Wrapper(model),
                                 folds,
                                 X, y, W=y, Z=None)
print(nuisance)
print(model_list)
fitted_inds

C:\ProgramData\Anaconda3\lib\site-packages\numba\errors.py:105: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


(array([-1.1057289 , -1.53756637, -2.4518278 , ...,  1.10628792,
       -1.82966233, -1.78227335]),)
[<__main__.Wrapper object at 0x0000015FD48EAD68>, <__main__.Wrapper object at 0x0000015FCF4B04A8>]


array([   0,    1,    2, ..., 4997, 4998, 4999])

# Simple DML with the _OrthoLearner

In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression
from econml._ortho_learner import _OrthoLearner
class ModelNuisance:
    def __init__(self, model_t, model_y):
        self._model_t = model_t
        self._model_y = model_y
    def fit(self, Y, T, W=None):
        self._model_t.fit(W, T)
        self._model_y.fit(W, Y)
        return self
    def predict(self, Y, T, W=None):
        return Y - self._model_y.predict(W), T - self._model_t.predict(W)
class ModelFinal:
    def __init__(self):
        return
    def fit(self, Y, T, W=None, nuisances=None):
        Y_res, T_res = nuisances
        self.model = LinearRegression(fit_intercept=False).fit(T_res.reshape(-1, 1), Y_res)
        return self
    def predict(self, X=None):
        return self.model.coef_[0]
    def score(self, Y, T, W=None, nuisances=None):
        Y_res, T_res = nuisances
        return np.mean((Y_res - self.model.predict(T_res.reshape(-1, 1)))**2)
np.random.seed(123)
X = np.random.normal(size=(100, 3))
y = X[:, 0] + X[:, 1] + np.random.normal(0, 0.1, size=(100,))
est = _OrthoLearner(ModelNuisance(LinearRegression(), LinearRegression()),
                    ModelFinal(),
                    n_splits=2, discrete_treatment=False, random_state=None)
est.fit(y, X[:, 0], W=X[:, 1:])

In [4]:
est.const_marginal_effect()

1.0236499258047584

In [5]:
est.effect(T0=0, T1=1)

array([1.02364993])

In [6]:
est.score(y, X[:, 0], W=X[:, 1:])

0.007279954240981791

In [7]:
est.model_final.model

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [8]:
est.model_final.model.coef_

array([1.02364993])

In [9]:
est.score_

0.007568302109999704

# Simple DML with Discrete Treatments with the _OrthoLearner

In [10]:
class ModelNuisance:
    def __init__(self, model_t, model_y):
        self._model_t = model_t
        self._model_y = model_y

    def fit(self, Y, T, W=None):
        self._model_t.fit(W, np.matmul(T, np.arange(1, T.shape[1]+1)))
        self._model_y.fit(W, Y)
        return self

    def predict(self, Y, T, W=None):
        return Y - self._model_y.predict(W), T - self._model_t.predict_proba(W)[:, 1:]

class ModelFinal:

    def __init__(self):
        return

    def fit(self, Y, T, W=None, nuisances=None):
        Y_res, T_res = nuisances
        self.model = LinearRegression(fit_intercept=False).fit(T_res.reshape(-1, 1), Y_res)
        return self

    def predict(self):
        # theta needs to be of dimension (1, d_t) if T is (n, d_t)
        return np.array([[self.model.coef_[0]]])

    def score(self, Y, T, W=None, nuisances=None):
        Y_res, T_res = nuisances
        return np.mean((Y_res - self.model.predict(T_res.reshape(-1, 1)))**2)

np.random.seed(123)
X = np.random.normal(size=(100, 3))
import scipy.special
from sklearn.linear_model import LogisticRegression
T = np.random.binomial(1, scipy.special.expit(X[:, 0]))
sigma = 0.01
y = T + X[:, 0] + np.random.normal(0, sigma, size=(100,))
est = _OrthoLearner(ModelNuisance(LogisticRegression(solver='lbfgs'), LinearRegression()), ModelFinal(),
                    n_splits=2, discrete_treatment=True, random_state=None)
est.fit(y, T, W=X)

In [11]:
est.const_marginal_effect()

array([[1.00123159]])

In [12]:
est.effect()

array([1.00123159])

In [13]:
est.score(y, T, W=X)

0.0025695883321466286

In [14]:
est.model_final.model.coef_[0]

1.0012315874866915

In [15]:
est.model_final.model

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [16]:
est.model_final.model.coef_

array([1.00123159])

In [17]:
est.score_

0.0031604059708364414

In [18]:
est.models_nuisance[0]._model_y.coef_

array([1.28171346, 0.03749846, 0.10120681])

# Simple DML with the _RLearner

In [19]:
import numpy as np
from sklearn.linear_model import LinearRegression
from econml._rlearner import _RLearner
from sklearn.base import clone
class ModelFirst:
    def __init__(self, model):
        self._model = clone(model, safe=False)
    def fit(self, X, W, Y, sample_weight=None):
        self._model.fit(np.hstack([X, W]), Y)
        return self
    def predict(self, X, W):
        return self._model.predict(np.hstack([X, W]))
class ModelFinal:
    def fit(self, X, T_res, Y_res, sample_weight=None, sample_var=None):
        self.model = LinearRegression(fit_intercept=False).fit(X * T_res.reshape(-1, 1), Y_res)
        return self
    def predict(self, X):
        return self.model.predict(X)
np.random.seed(123)
X = np.random.normal(size=(1000, 3))
y = X[:, 0] + X[:, 1] + np.random.normal(0, 0.01, size=(1000,))
est = _RLearner(ModelFirst(LinearRegression()),
                ModelFirst(LinearRegression()),
                ModelFinal(),
                n_splits=2, discrete_treatment=False, random_state=None)
est.fit(y, X[:, 0], X=np.ones((X.shape[0], 1)), W=X[:, 1:])

In [20]:
est.const_marginal_effect(np.ones((1,1)))

array([0.99963147])

In [21]:
est.effect(np.ones((1,1)), T0=0, T1=10)

array([9.99631472])

In [22]:
est.score(y, X[:, 0], X=np.ones((X.shape[0], 1)), W=X[:, 1:])

9.736380060274919e-05

In [23]:
est.model_final.model

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [24]:
est.model_final.model.coef_

array([0.99963147])

In [25]:
est.score_

9.826232040878185e-05

In [26]:
[mdl._model for mdl in est.models_y]

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False)]

In [27]:
[mdl._model for mdl in est.models_t]

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False)]

# Checking All Good with LinearDMLCateEstimator

In [28]:
from econml.dml import LinearDMLCateEstimator

np.random.seed(123)
X = np.random.normal(size=(1000, 3))
y = X[:, 0] + X[:, 1] + np.random.normal(0, 0.01, size=(1000,))
est = LinearDMLCateEstimator(model_y=LinearRegression(),
                             model_t=LinearRegression())
est.fit(y, X[:, 0], W=X[:, 1:], inference='statsmodels')

In [29]:
est.effect()

array([1.00033637])

In [30]:
est.effect_interval()

(array([0.99499205]), array([1.00568068]))

In [31]:
est.models_y

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False)]

In [32]:
est.models_t

[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False),
 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False)]

In [33]:
est.coef_

array([1.00033637])

In [34]:
est.coef__interval()

(array([0.99499205]), array([1.00568068]))

In [35]:
est.model_final

In [36]:
est.model_final._param_stderr

array([0.00324911])

# DR Learner Based on _OrthoLearner

In [44]:
import numpy as np
from sklearn.linear_model import LinearRegression
from econml._ortho_learner import _OrthoLearner
class ModelNuisance:
    def __init__(self, model_t, model_y):
        self._model_t = model_t
        self._model_y = model_y

    def fit(self, Y, T, X=None, W=None):
        self._model_t.fit(np.hstack([X, W]), np.matmul(T, np.arange(1, T.shape[1]+1)))
        self._model_y.fit(np.hstack([T, X, W]), Y)
        return self

    def predict(self, Y, T, X=None, W=None):
        propensities = self._model_t.predict_proba(np.hstack([X, W]))
        Y_pred = np.zeros((T.shape[0], T.shape[1] + 1))
        T_counter = np.zeros(T.shape)
        Y_pred[:, 0] = self._model_y.predict(np.hstack([T_counter, X, W]))
        Y_pred[:, 0] += (Y - Y_pred[:, 0]) * np.all(T==0, axis=1) / propensities[:, 0]
        for t in np.arange(T.shape[1]):
            T_counter = np.zeros(T.shape)
            T_counter[:, t] = 1
            Y_pred[:, t + 1] = self._model_y.predict(np.hstack([T_counter, X, W]))
            Y_pred[:, t + 1] += (Y - Y_pred[:, t + 1]) * (T[:, t] == 1) / propensities[:, t + 1]
        return Y_pred

class ModelFinal:

    def __init__(self):
        return

    def fit(self, Y, T, X=None, W=None, nuisances=None):
        Y_pred, = nuisances
        self.models_cate = [LinearRegression().fit(X, Y_pred[:, t] - Y_pred[:, 0])
                            for t in np.arange(1, Y_pred.shape[1])]
        return self

    def predict(self, X=None):
        # theta needs to be of dimension (1, d_t) if T is (n, d_t)
        return np.array([mdl.predict(X) for mdl in self.models_cate]).T

np.random.seed(123)
X = np.random.normal(size=(1000, 3))
import scipy.special
from sklearn.linear_model import LogisticRegression
T = np.random.binomial(1, scipy.special.expit(X[:, 0]))
sigma = 0.01
y = (1 + .5*X[:, 0]) * T + X[:, 0] + np.random.normal(0, sigma, size=(1000,))
est = _OrthoLearner(ModelNuisance(LogisticRegression(solver='lbfgs'), LinearRegression()), ModelFinal(),
                    n_splits=2, discrete_treatment=True, random_state=None)
est.fit(y, T, X=X[:, [0]], W=X[:, 1:])

In [45]:
est.const_marginal_effect(X[:10, [0]])

array([[ 0.43111746],
       [ 0.21377249],
       [-0.26176354],
       [ 0.54421168],
       [ 1.76258919],
       [ 0.76761463],
       [ 1.51079693],
       [ 1.76224943],
       [ 0.34418752],
       [ 0.2538734 ]])

In [46]:
[mdl._model_t.coef_ for mdl in est.models_nuisance]

[array([[ 1.07859458, -0.09378512, -0.16819498]]),
 array([[ 0.87520635, -0.07950399,  0.06037872]])]

In [47]:
[mdl._model_y.coef_ for mdl in est.models_nuisance]

[array([ 0.99703455,  1.25799456, -0.00554411,  0.00216083]),
 array([ 1.04547656e+00,  1.21020962e+00,  7.94069500e-04, -9.68240609e-03])]

In [48]:
est.model_final.models_cate[0].coef_

array([0.51667104])

In [49]:
est.model_final.models_cate[0].intercept_

0.9920313527587242

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
from econml.drlearner import DRLearner
import scipy.special
from sklearn.linear_model import LogisticRegression

np.random.seed(123)
X = np.random.normal(size=(1000, 3))
T = np.random.binomial(2, scipy.special.expit(X[:, 0]))
sigma = 0.001
y = (1 + .5*X[:, 0]) * T + X[:, 0] + np.random.normal(0, sigma, size=(1000,))
est = DRLearner()
est.fit(y, T, X=X, W=None)

In [3]:
est.effect(X[:2], T0=0, T1=1)

array([0.52446994, 0.37952909])

In [4]:
est.const_marginal_effect(X[:2])

array([[0.52446994, 0.8672687 ],
       [0.37952909, 0.27316111]])

In [5]:
[mdl.coef_ for mdl in est.models_regression]

[array([ 1.43597301e+00,  3.34210655e-04, -7.10298447e-03,  6.70792219e-01,
         1.98425625e+00]),
 array([ 1.49463372e+00, -2.46327311e-03,  2.00974603e-03,  6.82820433e-01,
         2.03497798e+00])]

In [6]:
[mdl.coef_ for mdl in est.models_propensity]

[array([[-1.03943214,  0.09104269,  0.112561  ],
        [ 0.08914442,  0.03477572, -0.09003806],
        [ 0.95028772, -0.12581841, -0.02252295]]),
 array([[-0.96593071,  0.09075624, -0.11942558],
        [ 0.04603771, -0.03461148, -0.07898284],
        [ 0.919893  , -0.05614476,  0.19840842]])]

In [7]:
est.model_cate(T=2).coef_

array([ 1.93280373,  1.06295052,  0.08980031, -0.00397677])

In [8]:
est.cate_feature_names

['1', 'x0', 'x1', 'x2']

In [9]:
est.model_cate(T=2).intercept_

0.0

In [10]:
est.model_cate(T=1).coef_

array([0.99869218, 0.46046774, 0.01178362, 0.04920265])

In [11]:
est.model_cate(T=1).intercept_

0.0

In [12]:
est.score_

9.29702483700037

In [13]:
est.score(y, T, X=X)

7.776002714350081

In [14]:
import numpy as np
from sklearn.linear_model import LinearRegression
from econml.drlearner import DRLearner, LinearDRLearner

np.random.seed(123)
X = np.random.normal(size=(1000, 3))
import scipy.special
from sklearn.linear_model import LogisticRegression
T = np.random.binomial(2, scipy.special.expit(X[:, 0]))
sigma = 0.01
y = (1 + .5*X[:, 0]) * T + X[:, 0] + np.random.normal(0, sigma, size=(1000,))
est = LinearDRLearner(LogisticRegression(solver='lbfgs'), LinearRegression())
est.fit(y, T, X=X, W=None, inference='statsmodels')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [15]:
from econml.utilities import StatsModelsLinearRegression

In [16]:
est.effect_interval(X[:3])

(array([ 0.4087843 ,  0.16615335, -0.32221422]),
 array([0.59866562, 0.5106094 , 0.14135485]))

In [17]:
low, up = est.const_marginal_effect_interval(X[:3])

In [18]:
point = est.const_marginal_effect(X[:3])

In [19]:
for x, l, p, u in zip(X, low, point, up):
    print("X={}".format(x))
    for it, (ll, pp, uu) in enumerate(zip(l, p, u)):
        print(ll, pp, uu)
        print((1 + .5*x[0])*(it+1))

X=[-1.0856306   0.99734545  0.2829785 ]
0.4087843003398282 0.5037249622393646 0.5986656241389009
0.4571846983497194
0.6267599463600815 1.012628073862441 1.3984962013648001
0.9143693966994388
X=[-1.50629471 -0.57860025  1.65143654]
0.1661533536426545 0.33838137530437185 0.5106093969660891
0.246852643040954
-0.07764920507354223 0.5123144081449562 1.1022780213634544
0.493705286081908
X=[-2.42667924 -0.42891263  1.26593626]
-0.32221422472280065 -0.09042968751747016 0.14135484968786025
-0.21333962169653709
-1.1461891997437945 -0.3757694761362078 0.39465024747137845
-0.42667924339307417


In [20]:
est.coef__interval(T=2)

(array([ 1.82303692,  0.71414488, -0.04264683, -0.1179687 ]),
 array([2.14002152, 1.22322968, 0.1967088 , 0.15973097]))